In [16]:
import nltk
# nltk.download()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words('english'))
print(stop_words)

example_sentence = "This is an example showing off stop word filtering but we need conjunctions."
words = word_tokenize(example_sentence)
filtered_sentence = [w for w in words if not w in stop_words]
print(filtered_sentence)

ex1 = "I didn't know, You are, He is, She is, We are, They are"
ps = PorterStemmer()
words1 = word_tokenize(ex1)
for w in words1:
    print(ps.stem(w))

{'shouldn', 'not', 'yours', 'mustn', 'was', "didn't", 'being', "wasn't", "you've", 'from', 'below', 'few', 'your', 'been', "you'd", 'am', 'more', 'this', 'it', "hasn't", 'here', 'only', 'about', 'mightn', 'did', 'than', 'd', 'aren', "you're", 'nor', "haven't", "should've", 'hasn', 'under', 'you', 'needn', 'our', 'itself', 'o', "she's", 'again', 'very', 'into', 'with', 'can', 'wouldn', 'herself', 'me', "shouldn't", 'own', 'and', 'i', 'no', 'he', 'then', 'had', 'my', "that'll", 'ours', 'wasn', "hadn't", 'whom', 'after', 'further', 'of', "don't", 'when', "mustn't", 'on', 'y', 'him', 'until', 'each', 'ma', 'as', 'against', 'out', "couldn't", "weren't", 're', 'is', 'during', 'yourself', 'his', 'now', 'down', 'doing', 'but', 'were', "won't", 'if', 'hers', "needn't", 'her', 'do', 'by', 'above', 'at', 'don', 'that', 'have', 'before', 'ourselves', 'to', 'himself', 'in', 'why', 's', 'the', 'll', 'weren', 'an', "it's", 'couldn', 'doesn', 'isn', 'which', 'too', 'what', "doesn't", 'once', 'just', '

In [1]:
import gzip, json, pandas as pd
from sklearn.model_selection import train_test_split

input = '../Data/reviews_Cell_Phones_and_Accessories_5.json.gz'
data = []
with gzip.open(input) as f:
    for l in f:
        data.append(json.loads(l.strip()))

data = pd.DataFrame.from_dict(data)
target = data['overall']
text = data['reviewText']

# from collections import Counter
# print(sorted(Counter(target).items()))

In [56]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_res, y_res = ros.fit_resample(data[['reviewText','overall']], data['overall'])
print(sorted(Counter(y_res).items()))

[(1.0, 108664), (2.0, 108664), (3.0, 108664), (4.0, 108664), (5.0, 108664)]


In [57]:
print(X_res.iloc[:3]['reviewText'])

0    They look good and stick good! I just don't li...
1    These stickers work like the review says they ...
2    These are awesome and make my phone look so st...
Name: reviewText, dtype: object


In [61]:
from nltk import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

lemmatizer = WordNetLemmatizer()
print(text[0])
words = word_tokenize(text[0])
print(words)
# stemmed = [ps.stem(w) for w in words if not w in stop_words]
stemmed = [lemmatizer.lemmatize(w) for w in words if not w in stop_words]
str1 = TreebankWordDetokenizer().detokenize(stemmed)
print(stemmed)
print(str1)
print(target[0])
t1 = "I like apples, but grapes I like best. I don't like pears"

They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again
['They', 'look', 'good', 'and', 'stick', 'good', '!', 'I', 'just', 'do', "n't", 'like', 'the', 'rounded', 'shape', 'because', 'I', 'was', 'always', 'bumping', 'it', 'and', 'Siri', 'kept', 'popping', 'up', 'and', 'it', 'was', 'irritating', '.', 'I', 'just', 'wo', "n't", 'buy', 'a', 'product', 'like', 'this', 'again']
['They', 'look', 'good', 'stick', 'good', '!', 'I', "n't", 'like', 'rounded', 'shape', 'I', 'always', 'bumping', 'Siri', 'kept', 'popping', 'irritating', '.', 'I', 'wo', "n't", 'buy', 'product', 'like']
They look good stick good! In't like rounded shape I always bumping Siri kept popping irritating . I won't buy product like
4.0


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X_res['reviewText'], y_res, test_size=0.25, random_state=109)
print(sorted(Counter(y_train).items()))
print(sorted(Counter(y_test).items()))

import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    return reviews

reviews_train_clean = preprocess_reviews(X_train)
reviews_test_clean = preprocess_reviews(X_test)
# target = df['overall'].values.tolist()
# text = df['reviewText'].values.tolist()

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
final_model = LogisticRegression(C=0.05)
final_model.fit(X, y_train)
print ("Final Accuracy: %s"
       % accuracy_score(y_test, final_model.predict(X_test)))
print ("Final F1: %s"
       % f1_score(y_test, final_model.predict(X_test),average='weighted'))
# for test size = 0.25
# Final Accuracy: 0.6485908249331414
# Final F1: 0.6144467230121968
# balanced with under sampling:
# Final Accuracy: 0.5862168278132072
# Final F1: 0.6035250426507991
# balanced with over sampling:
# Final Accuracy: 0.600228226459545
# Final F1: 0.5971548939970822

c:\users\d064966\appdata\local\continuum\anaconda3\envs\masterthesis\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Final Accuracy: 0.600228226459545
Final F1: 0.5971548939970822


In [47]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
print(reviews_train_clean[0])
words = word_tokenize(reviews_train_clean[0])
stemmed = [ps.stem(w) for w in words]
str1 = TreebankWordDetokenizer().detokenize(stemmed)
print(stemmed)
print(str1)
print(y_train[0])

In [ ]:
df = pd.DataFrame()
cv = CountVectorizer(binary=True)
cv.fit(text)
text_cv = cv.fit_transform(text)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer()
tfidfv.fit(text)
text_tfidf = tfidfv.fit_transform(text)


In [35]:
print(text[0])

They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again


In [33]:
print(text_cv[0])

  (0, 8056)	1
  (0, 8840)	1
  (0, 9240)	1
  (0, 13265)	1
  (0, 16087)	1
  (0, 16398)	1
  (0, 26912)	1
  (0, 35530)	1
  (0, 42602)	1
  (0, 42844)	1
  (0, 43857)	1
  (0, 44102)	1
  (0, 45927)	1
  (0, 46639)	1
  (0, 59045)	1
  (0, 60527)	1
  (0, 65967)	1
  (0, 68457)	1
  (0, 69545)	1
  (0, 72953)	1
  (0, 76274)	1
  (0, 77004)	1
  (0, 77199)	1
  (0, 81243)	1
  (0, 83530)	1
  (0, 85176)	1


In [37]:
print(text_tfidf.shape, text_cv.shape)

(194439, 86719) (194439, 86719)


In [51]:
df1 = pd.DataFrame()
df1['countvector'] = list(text_cv)
df1['tdidfvector'] = list(text_tfidf)
df1.head()

,countvector,tdidfvector
0,"(0, 77004)\t1\n (0, 46639)\t1\n (0, 35530)...","(0, 8056)\t0.15507078525237655\n (0, 77199)..."
1,"(0, 77004)\t1\n (0, 9240)\t1\n (0, 72953)\...","(0, 69566)\t0.2926513088281011\n (0, 51151)..."
2,"(0, 46639)\t1\n (0, 9240)\t1\n (0, 42844)\...","(0, 62028)\t0.16355481816851297\n (0, 76206..."
3,"(0, 77004)\t1\n (0, 9240)\t1\n (0, 13265)\...","(0, 20413)\t0.15212280316459825\n (0, 19417..."
4,"(0, 9240)\t1\n (0, 42844)\t1\n (0, 54028)\...","(0, 44133)\t0.29373757250840965\n (0, 28874..."


In [122]:
import numpy as np
df3 = data['reviewText'][:45000]
X = list(cv.fit_transform(df3).toarray())
Y = target[:45000]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, shuffle= True)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
final_model = LogisticRegression(C=0.05)

MemoryError: Unable to allocate 13.6 GiB for an array with shape (45000, 40560) and data type int64

In [56]:
print(X_train.head())

                                              countvector  \
111639    (0, 9240)\t1\n  (0, 76274)\t1\n  (0, 42844)\...   
32789     (0, 9240)\t1\n  (0, 26912)\t1\n  (0, 76274)\...   
186518    (0, 9240)\t1\n  (0, 76274)\t1\n  (0, 42844)\...   
110683    (0, 77199)\t1\n  (0, 36080)\t1\n  (0, 33222)...   
151007    (0, 9240)\t1\n  (0, 42844)\t1\n  (0, 17047)\...   

                                              tdidfvector  
111639    (0, 65557)\t0.3559509384448291\n  (0, 37270)...  
32789     (0, 12296)\t0.24388994197069008\n  (0, 73506...  
186518    (0, 12735)\t0.2475585090448447\n  (0, 82444)...  
110683    (0, 82730)\t0.46573119423546444\n  (0, 5576)...  
151007    (0, 24106)\t0.3641800628526834\n  (0, 35402)...  


In [73]:
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s"
       % accuracy_score(y_test, final_model.predict(X_test)))
print ("Final F1: %s"
       % f1_score(y_test, final_model.predict(X_test),average='weighted'))
# With Countvectorizer von 10000 first data entries
# Final Accuracy: 0.5875757575757575
# Final F1: 0.5483412446258874

c:\users\d064966\appdata\local\continuum\anaconda3\envs\masterthesis\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Final Accuracy: 0.5875757575757575
Final F1: 0.5483412446258874


In [127]:
from sklearn_pandas import DataFrameMapper
print(type(text))
df4 = pd.concat([text[:30000], text[:30000]], axis=1)
print(df4.shape)
df4.columns=['ersterText', 'zweiterText']
print(df4.describe())

mapper = DataFrameMapper([
    ('ersterText', CountVectorizer()),
    ('zweiterText', TfidfVectorizer())
])

features = mapper.fit_transform(df4)

x, x_test, y, y_test = train_test_split(features,target[:10000],test_size=0.2,train_size=0.8, random_state = 0)

lR = LogisticRegression(C=0.05)
lR.fit(x, y)
predicted = lR.predict(x_test)
print ("Final Accuracy: %s"
       % accuracy_score(y_test, predicted))
print ("Final F1: %s"
       % f1_score(y_test, predicted,average='weighted'))

<class 'pandas.core.series.Series'>
(30000, 2)
       ersterText zweiterText
count       30000       30000
unique      29982       29982
top                          
freq           17          17


MemoryError: Unable to allocate 15.1 GiB for an array with shape (30000, 67394) and data type float64

In [118]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)
print(features[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [ ]:
# text_list = text.values.tolist()
# target_list = target.values.tolist()
# text_list[:3]

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)
final_model = LogisticRegression(C=0.05)
final_model.fit(X, y_train)


c:\users\d064966\appdata\local\continuum\anaconda3\envs\masterthesis\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=0.05)

In [34]:
print(X[:3])

  (0, 4318)	1
  (0, 6811)	1
  (0, 7181)	1
  (0, 15506)	1
  (0, 20573)	1
  (0, 29220)	1
  (0, 30203)	1
  (0, 31509)	1
  (0, 33449)	1
  (0, 56516)	1
  (0, 60278)	1
  (0, 61061)	1
  (0, 61979)	1
  (0, 66734)	1
  (0, 85720)	1
  (0, 89797)	1
  (0, 90791)	1
  (0, 91388)	1
  (0, 103743)	1
  (0, 105980)	1
  (0, 115424)	1
  (0, 115724)	1
  (0, 129273)	1
  (1, 3495)	1
  (1, 4318)	1
  :	:
  (2, 6811)	1
  (2, 8683)	1
  (2, 10507)	1
  (2, 15909)	1
  (2, 19191)	1
  (2, 19798)	1
  (2, 20545)	1
  (2, 23724)	1
  (2, 33483)	1
  (2, 42409)	1
  (2, 44718)	1
  (2, 51706)	1
  (2, 52937)	1
  (2, 72453)	1
  (2, 75478)	1
  (2, 75948)	1
  (2, 78732)	1
  (2, 95597)	1
  (2, 102188)	1
  (2, 113166)	1
  (2, 117292)	1
  (2, 119495)	1
  (2, 125184)	1
  (2, 130987)	1
  (2, 131495)	1


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer()
tfidfv.fit(reviews_train_clean)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c)
    lr.fit(X, y_train)
    print ("F1 for C=%s: %s"
           % (c, f1_score(y_test, lr.predict(X_test),average='weighted')))
# micro:
# F1 for C=0.01: 0.6441267228965234
# F1 for C=0.05: 0.6485908249331414
# F1 for C=0.25: 0.6444147294795309
# F1 for C=0.5: 0.6429541246657067
# F1 for C=1: 0.6412672289652335
# weighted
# F1 for C=0.01: 0.6010131542198381
# F1 for C=0.05: 0.6144467230121968
# F1 for C=0.25: 0.6152861961759271
# F1 for C=0.5: 0.6169158482680683
# F1 for C=1: 0.614103362804607


for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c)
    lr.fit(X, y_train)
    print ("Accuracy for C=%s: %s"
           % (c, accuracy_score(y_test, lr.predict(X_test))))
# Accuracy for C=0.01: 0.6441267228965234
# Accuracy for C=0.05: 0.6485908249331414
# Accuracy for C=0.25: 0.6444147294795309
# Accuracy for C=0.5: 0.6429541246657067
# Accuracy for C=1: 0.6412672289652335